<a href="https://colab.research.google.com/github/manuelarguelles/fundamentos_python_y_limpieza_101/blob/main/Procesamiento_Lotes_Series_C_2301.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de Indicadores Financieros - Cajas Rurales de Ahorro y Crédito

## Objetivo
Transformar el archivo Excel de indicadores financieros de formato wide (una columna por CRAC) a formato long (normalizado) con estructura:
- `periodo`: Fecha del reporte
- `crac`: Nombre de la caja rural
- `categoria`: Categoría del indicador (SOLVENCIA, CALIDAD, etc.)
- `indicador`: Nombre específico del indicador
- `valor`: Valor numérico

---

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import re
from datetime import datetime

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


---
## 1. Configuración de Ruta y Listado de Archivos

Procesaremos todos los archivos Excel (.xls, .xlsx) en la carpeta especificada.

In [ ]:
import glob
import os

# Configurar la ruta donde están los archivos Excel
# Cambiar esta ruta según tu estructura de carpetas
ruta_archivos = '/content/'  # Ruta por defecto en Colab

# Alternativamente, si montaste Google Drive:
# ruta_archivos = '/content/drive/MyDrive/tu_carpeta/'

# Listar todos los archivos Excel en la ruta
extensiones = ['*.xls', '*.xlsx', '*.XLS', '*.XLSX']
archivos_excel = []

for extension in extensiones:
    archivos_excel.extend(glob.glob(os.path.join(ruta_archivos, extension)))

# Ordenar archivos alfabéticamente
archivos_excel.sort()

print(f"=== ARCHIVOS EXCEL ENCONTRADOS ===")
print(f"Ruta: {ruta_archivos}")
print(f"Total de archivos: {len(archivos_excel)}\n")

if len(archivos_excel) == 0:
    print("⚠️ No se encontraron archivos Excel en la ruta especificada")
    print("Verifica que la ruta sea correcta y contenga archivos .xls o .xlsx")
else:
    print("Archivos a procesar:")
    for i, archivo in enumerate(archivos_excel, 1):
        nombre_archivo = os.path.basename(archivo)
        tamaño = os.path.getsize(archivo) / 1024  # KB
        print(f"  {i}. {nombre_archivo} ({tamaño:.1f} KB)")

=== ARCHIVOS EXCEL ENCONTRADOS ===
Ruta: /content/
Total de archivos: 19

Archivos a procesar:
  1. C-2301-ab2024.XLS (48.0 KB)
  2. C-2301-ab2025.XLS (48.0 KB)
  3. C-2301-ag2024.XLS (48.0 KB)
  4. C-2301-di2024.XLS (38.5 KB)
  5. C-2301-en2024.XLS (47.5 KB)
  6. C-2301-en2025.XLS (48.0 KB)
  7. C-2301-fe2024.XLS (48.0 KB)
  8. C-2301-fe2025.XLS (48.0 KB)
  9. C-2301-jl2024.XLS (48.0 KB)
  10. C-2301-jl2025.XLS (47.5 KB)
  11. C-2301-jn2024.XLS (47.5 KB)
  12. C-2301-jn2025.XLS (47.0 KB)
  13. C-2301-ma2024.XLS (47.5 KB)
  14. C-2301-ma2025.XLS (48.0 KB)
  15. C-2301-my2024.XLS (48.0 KB)
  16. C-2301-my2025.XLS (45.5 KB)
  17. C-2301-no2024.XLS (48.0 KB)
  18. C-2301-oc2024.XLS (48.0 KB)
  19. C-2301-se2024.XLS (48.0 KB)


---
## 2. Función de Procesamiento de Archivo Individual

Creamos una función que procesa un archivo Excel y retorna el DataFrame normalizado.

In [ ]:
def procesar_archivo_crac(ruta_archivo):
    """
    Procesa un archivo Excel de indicadores CRAC y retorna DataFrame normalizado

    Parámetros:
    -----------
    ruta_archivo : str
        Ruta completa al archivo Excel

    Retorna:
    --------
    pd.DataFrame : DataFrame normalizado con columnas [periodo, crac, categoria, indicador, valor]
    """

    nombre_archivo = os.path.basename(ruta_archivo)
    print(f"\n{'='*60}")
    print(f"Procesando: {nombre_archivo}")
    print('='*60)

    try:
        # 1. Leer archivo completo para extraer período
        df_raw = pd.read_excel(ruta_archivo, header=None)

        # 2. Extraer período de la celda A3 (índice 2)
        periodo_texto = df_raw.iloc[2, 0]
        print(f"Período extraído: {periodo_texto}")
        print(f"Tipo: {type(periodo_texto)}")

        # Función para parsear fecha (maneja datetime y string)
        def extraer_periodo(texto):
            # Caso 1: Ya es un objeto datetime
            if isinstance(texto, datetime):
                print("  → Ya es datetime, usando directamente")
                return texto

            # Caso 2: Es pd.Timestamp
            if isinstance(texto, pd.Timestamp):
                print("  → Es pd.Timestamp, convirtiendo a datetime")
                return texto.to_pydatetime()

            # Caso 3: Es string, parsear
            if isinstance(texto, str):
                print("  → Es string, parseando...")
                meses = {
                    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
                    'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
                    'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
                }
                patron = r'(\d+)\s+de\s+(\w+)\s+de\s+(\d{4})'
                match = re.search(patron, texto.lower())

                if match:
                    dia = int(match.group(1))
                    mes_texto = match.group(2)
                    año = int(match.group(3))
                    mes = meses.get(mes_texto, 1)
                    return datetime(año, mes, dia)

            # Si no se pudo extraer, intentar conversión directa
            try:
                return pd.to_datetime(texto)
            except:
                return None

        periodo = extraer_periodo(periodo_texto)

        if periodo is None:
            print(f"⚠️ No se pudo extraer el período del archivo")
            return None

        print(f"  → Período final: {periodo.strftime('%Y-%m-%d')}")

        # 3. Leer con headers correctos (fila 5, índice 4)
        df = pd.read_excel(ruta_archivo, header=4)

        # Limpiar nombres de columnas (eliminar saltos de línea y espacios extras)
        df.columns = df.columns.map(lambda x: str(x).replace('\n', ' ').replace('\r', ' ').strip())
        df.columns = df.columns.map(lambda x: ' '.join(x.split()))  # Eliminar espacios múltiples

        print(f"  → Columnas después de limpieza:")
        for col in df.columns:
            print(f"     • {col}")

        # 4. Buscar fila de "Nota" y filtrar
        fila_nota = None
        for idx, row in df.iterrows():
            if pd.notna(row.iloc[0]) and 'Nota' in str(row.iloc[0]):
                fila_nota = idx
                break

        if fila_nota is not None:
            df = df.iloc[:fila_nota].copy()
            print(f"  → Datos hasta fila {fila_nota} (antes de notas)")

        # 5. Renombrar primera columna
        df.columns.values[0] = 'indicador'

        # 6. Identificar columnas de CRACs (excluyendo las que empiezan con "Unnamed")
        columnas_cracs = [col for col in df.columns if col != 'indicador' and not col.startswith('Unnamed')]
        print(f"  → CRACs válidas encontradas: {len(columnas_cracs)}")

        # *** NUEVO: Eliminar columnas Unnamed antes del procesamiento ***
        columnas_a_mantener = ['indicador'] + columnas_cracs
        df = df[columnas_a_mantener].copy()

        # Actualizar lista de columnas CRACs después de filtrar
        columnas_cracs = [col for col in df.columns if col != 'indicador']

        # 7. Identificar y marcar categorías
        def es_categoria(row):
            tiene_texto = pd.notna(row['indicador']) and str(row['indicador']).strip() != ''
            valores_vacios = all(pd.isna(row[col]) or str(row[col]).strip() == ''
                               for col in columnas_cracs)
            return tiene_texto and valores_vacios

        df['es_categoria'] = df.apply(es_categoria, axis=1)
        categorias_encontradas = df[df['es_categoria']]['indicador'].tolist()
        print(f"  → Categorías encontradas: {len(categorias_encontradas)}")

        # 8. Asignar categorías (forward fill)
        df['categoria'] = None
        categoria_actual = None
        for idx, row in df.iterrows():
            if row['es_categoria']:
                categoria_actual = row['indicador']
            df.at[idx, 'categoria'] = categoria_actual

        # 9. Eliminar filas de categorías y vacías
        registros_antes = len(df)
        df = df[~df['es_categoria']].copy()
        df = df[pd.notna(df['indicador'])].copy()
        df = df[df['indicador'].astype(str).str.strip() != ''].copy()
        df = df.drop('es_categoria', axis=1)
        print(f"  → Registros de indicadores: {len(df)} (eliminados {registros_antes - len(df)} de categorías/vacíos)")

        # 10. Limpiar nombres (categorías e indicadores)
        df['categoria'] = df['categoria'].str.replace('*', '', regex=False).str.strip()
        df['indicador'] = df['indicador'].str.strip()

        # Limpiar saltos de línea en indicadores
        df['indicador'] = df['indicador'].str.replace('\n', ' ', regex=False)
        df['indicador'] = df['indicador'].str.replace('\r', ' ', regex=False)
        df['indicador'] = df['indicador'].str.replace(r'\s+', ' ', regex=True)  # Espacios múltiples

        # 11. Pivotear a formato long
        df_long = df.melt(
            id_vars=['categoria', 'indicador'],
            var_name='crac',
            value_name='valor'
        )
        print(f"  → Después de pivoteo: {len(df_long)} registros")

        # 12. Limpiar valores numéricos
        def limpiar_valor(valor):
            if isinstance(valor, (int, float)):
                return float(valor)
            if pd.isna(valor):
                return np.nan

            valor_str = str(valor).strip()

            if valor_str == '-':
                return 0.0

            if '(' in valor_str:
                valor_str = valor_str.replace('(', '').replace(')', '').strip()
                try:
                    return -float(valor_str)
                except:
                    return np.nan

            try:
                valor_str = valor_str.replace(' ', '').replace(',', '')
                return float(valor_str)
            except:
                return np.nan

        df_long['valor'] = df_long['valor'].apply(limpiar_valor)

        # 13. Agregar columna de período
        df_long['periodo'] = periodo

        # 14. Reordenar columnas
        df_final = df_long[['periodo', 'crac', 'categoria', 'indicador', 'valor']].copy()
        df_final = df_final.sort_values(['categoria', 'indicador', 'crac']).reset_index(drop=True)

        print(f"✓ Procesado exitosamente: {len(df_final)} registros")
        print(f"  - Categorías: {df_final['categoria'].nunique()}")
        print(f"  - Indicadores: {df_final['indicador'].nunique()}")
        print(f"  - CRACs: {df_final['crac'].nunique()}")

        return df_final

    except Exception as e:
        print(f"✗ Error procesando archivo: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

print("✓ Función de procesamiento creada (sin columnas Unnamed)")

✓ Función de procesamiento creada (sin columnas Unnamed)


---
## 3. Procesamiento de Todos los Archivos

Iteramos sobre todos los archivos encontrados y consolidamos los resultados.

In [ ]:
# Procesar todos los archivos
dataframes_procesados = []
archivos_exitosos = []
archivos_fallidos = []

for archivo in archivos_excel:
    df_resultado = procesar_archivo_crac(archivo)

    if df_resultado is not None:
        dataframes_procesados.append(df_resultado)
        archivos_exitosos.append(os.path.basename(archivo))
    else:
        archivos_fallidos.append(os.path.basename(archivo))

print(f"\n{'='*60}")
print("RESUMEN DE PROCESAMIENTO")
print('='*60)
print(f"Total archivos procesados: {len(archivos_excel)}")
print(f"Exitosos: {len(archivos_exitosos)}")
print(f"Fallidos: {len(archivos_fallidos)}")

if archivos_fallidos:
    print(f"\n⚠️ Archivos con errores:")
    for archivo in archivos_fallidos:
        print(f"  - {archivo}")


Procesando: C-2301-ab2024.XLS
Período extraído: 2024-04-30 00:00:00
Tipo: <class 'datetime.datetime'>
  → Ya es datetime, usando directamente
  → Período final: 2024-04-30
  → Columnas después de limpieza:
     • Unnamed: 0
     • CRAC Cencosud Scotia
     • Unnamed: 2
     • CRAC Los Andes
     • CRAC Prymera
     • Unnamed: 5
     • Unnamed: 6
     • Unnamed: 7
     • Unnamed: 8
     • Unnamed: 9
     • CRAC Incasur
     • CRAC del Centro
     • TOTAL CRAC
  → Datos hasta fila 32 (antes de notas)
  → CRACs válidas encontradas: 6
  → Categorías encontradas: 6
  → Registros de indicadores: 20 (eliminados 12 de categorías/vacíos)
  → Después de pivoteo: 120 registros
✓ Procesado exitosamente: 120 registros
  - Categorías: 6
  - Indicadores: 20
  - CRACs: 6

Procesando: C-2301-ab2025.XLS
Período extraído: 2025-04-30 00:00:00
Tipo: <class 'datetime.datetime'>
  → Ya es datetime, usando directamente
  → Período final: 2025-04-30
  → Columnas después de limpieza:
     • Unnamed: 0
     • C

In [ ]:
# Consolidar todos los DataFrames en uno solo
if len(dataframes_procesados) > 0:
    df_consolidado = pd.concat(dataframes_procesados, ignore_index=True)

    print(f"\n=== DATASET CONSOLIDADO ===")
    print(f"Total de registros: {len(df_consolidado):,}")
    print(f"Períodos únicos: {df_consolidado['periodo'].nunique()}")
    print(f"CRACs únicos: {df_consolidado['crac'].nunique()}")
    print(f"Categorías: {df_consolidado['categoria'].nunique()}")
    print(f"Indicadores únicos: {df_consolidado['indicador'].nunique()}")

    print(f"\n--- Períodos en el dataset ---")
    for periodo in sorted(df_consolidado['periodo'].unique()):
        registros = len(df_consolidado[df_consolidado['periodo'] == periodo])
        print(f"  • {periodo.strftime('%Y-%m-%d')}: {registros:,} registros")

    print(f"\n--- CRACs en el dataset ---")
    for crac in df_consolidado['crac'].unique():
        registros = len(df_consolidado[df_consolidado['crac'] == crac])
        print(f"  • {crac}: {registros:,} registros")

else:
    print("⚠️ No se procesó ningún archivo exitosamente")
    df_consolidado = None


=== DATASET CONSOLIDADO ===
Total de registros: 1,920
Períodos únicos: 16
CRACs únicos: 6
Categorías: 6
Indicadores únicos: 20

--- Períodos en el dataset ---
  • 2024-01-31: 120 registros
  • 2024-02-29: 120 registros
  • 2024-03-31: 120 registros
  • 2024-04-30: 120 registros
  • 2024-05-31: 120 registros
  • 2024-07-31: 120 registros
  • 2024-08-31: 120 registros
  • 2024-09-30: 120 registros
  • 2024-10-31: 120 registros
  • 2024-11-30: 120 registros
  • 2024-12-31: 120 registros
  • 2025-01-31: 120 registros
  • 2025-02-28: 120 registros
  • 2025-03-31: 120 registros
  • 2025-04-30: 120 registros
  • 2025-07-31: 120 registros

--- CRACs en el dataset ---
  • CRAC Cencosud Scotia: 320 registros
  • CRAC Incasur: 320 registros
  • CRAC Los Andes: 320 registros
  • CRAC Prymera: 320 registros
  • CRAC del Centro: 320 registros
  • TOTAL CRAC: 320 registros


In [ ]:
# Vista previa del dataset consolidado
if df_consolidado is not None:
    print("Primeras 30 filas del dataset consolidado:")
    print(df_consolidado.head(30))

    print("\n\nÚltimas 30 filas:")
    print(df_consolidado.tail(30))

Primeras 30 filas del dataset consolidado:
      periodo                  crac           categoria  \
0  2024-04-30  CRAC Cencosud Scotia  CALIDAD DE ACTIVOS   
1  2024-04-30          CRAC Incasur  CALIDAD DE ACTIVOS   
2  2024-04-30        CRAC Los Andes  CALIDAD DE ACTIVOS   
3  2024-04-30          CRAC Prymera  CALIDAD DE ACTIVOS   
4  2024-04-30       CRAC del Centro  CALIDAD DE ACTIVOS   
5  2024-04-30            TOTAL CRAC  CALIDAD DE ACTIVOS   
6  2024-04-30  CRAC Cencosud Scotia  CALIDAD DE ACTIVOS   
7  2024-04-30          CRAC Incasur  CALIDAD DE ACTIVOS   
8  2024-04-30        CRAC Los Andes  CALIDAD DE ACTIVOS   
9  2024-04-30          CRAC Prymera  CALIDAD DE ACTIVOS   
10 2024-04-30       CRAC del Centro  CALIDAD DE ACTIVOS   
11 2024-04-30            TOTAL CRAC  CALIDAD DE ACTIVOS   
12 2024-04-30  CRAC Cencosud Scotia  CALIDAD DE ACTIVOS   
13 2024-04-30          CRAC Incasur  CALIDAD DE ACTIVOS   
14 2024-04-30        CRAC Los Andes  CALIDAD DE ACTIVOS   
15 2024-04-30

---
## 4. Validaciones de Integridad

Validamos que la columna TOTAL CRAC sea la suma de las demás CRACs para cada período.

In [ ]:
# Validación de integridad por período
if df_consolidado is not None:

    for periodo_val in sorted(df_consolidado['periodo'].unique()):

        print(f"\n{'='*60}")
        print(f"VALIDACIÓN PARA PERÍODO: {periodo_val.strftime('%Y-%m-%d')}")
        print('='*60)

        df_periodo = df_consolidado[df_consolidado['periodo'] == periodo_val]

        # Identificar CRACs
        cracs_individuales = [c for c in df_periodo['crac'].unique() if 'TOTAL' not in c.upper()]
        crac_total_list = [c for c in df_periodo['crac'].unique() if 'TOTAL' in c.upper()]

        if not crac_total_list:
            print("⚠️ No se encontró columna TOTAL CRAC en este período")
            continue

        crac_total = crac_total_list[0]

        validaciones = []

        for categoria in df_periodo['categoria'].unique():
            for indicador in df_periodo[df_periodo['categoria'] == categoria]['indicador'].unique():

                valores_individuales = df_periodo[
                    (df_periodo['categoria'] == categoria) &
                    (df_periodo['indicador'] == indicador) &
                    (df_periodo['crac'].isin(cracs_individuales))
                ]['valor']

                valor_total = df_periodo[
                    (df_periodo['categoria'] == categoria) &
                    (df_periodo['indicador'] == indicador) &
                    (df_periodo['crac'] == crac_total)
                ]['valor'].values

                if len(valor_total) > 0:
                    suma_calculada = valores_individuales.sum()
                    total_reportado = valor_total[0]

                    if not np.isnan(suma_calculada) and not np.isnan(total_reportado):
                        diferencia = abs(suma_calculada - total_reportado)
                        es_valido = diferencia < 0.1

                        validaciones.append({
                            'categoria': categoria,
                            'indicador': indicador,
                            'suma_calculada': round(suma_calculada, 2),
                            'total_reportado': round(total_reportado, 2),
                            'diferencia': round(diferencia, 2),
                            'valido': es_valido
                        })

        df_val = pd.DataFrame(validaciones)

        if len(df_val) > 0:
            print(f"Validaciones realizadas: {len(df_val)}")
            print(f"Exitosas: {df_val['valido'].sum()}")
            print(f"Fallidas: {(~df_val['valido']).sum()}")

            if (~df_val['valido']).any():
                print("\n⚠️ Validaciones fallidas:")
                print(df_val[~df_val['valido']][['categoria', 'indicador', 'diferencia']])
            else:
                print("✓ Todas las validaciones pasaron")


VALIDACIÓN PARA PERÍODO: 2024-01-31
Validaciones realizadas: 20
Exitosas: 0
Fallidas: 20

⚠️ Validaciones fallidas:
                        categoria  \
0              CALIDAD DE ACTIVOS   
1              CALIDAD DE ACTIVOS   
2              CALIDAD DE ACTIVOS   
3              CALIDAD DE ACTIVOS   
4              CALIDAD DE ACTIVOS   
5              CALIDAD DE ACTIVOS   
6            EFICIENCIA Y GESTIÓN   
7            EFICIENCIA Y GESTIÓN   
8            EFICIENCIA Y GESTIÓN   
9            EFICIENCIA Y GESTIÓN   
10           EFICIENCIA Y GESTIÓN   
11           EFICIENCIA Y GESTIÓN   
12                       LIQUIDEZ   
13                       LIQUIDEZ   
14                       LIQUIDEZ   
15  POSICIÓN EN MONEDA EXTRANJERA   
16                   RENTABILIDAD   
17                   RENTABILIDAD   
18                      SOLVENCIA   
19                      SOLVENCIA   

                                            indicador  diferencia  
0      Cartera de Alto Riesgo / Crédi

---
## 5. Exportar Dataset Consolidado

In [ ]:
# Guardar dataset consolidado
if df_consolidado is not None:
    nombre_archivo_salida = 'indicadores_financieros_crac_consolidado.csv'
    df_consolidado.to_csv(nombre_archivo_salida, index=False)

    print(f"✓ Dataset consolidado exportado a: {nombre_archivo_salida}")
    print(f"  Registros totales: {len(df_consolidado):,}")
    print(f"  Tamaño del archivo: {os.path.getsize(nombre_archivo_salida) / 1024:.1f} KB")

    # Descargar archivo
    from google.colab import files
    files.download(nombre_archivo_salida)

✓ Dataset consolidado exportado a: indicadores_financieros_crac_consolidado.csv
  Registros totales: 1,920
  Tamaño del archivo: 203.9 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>